In [17]:
import tensorflow as tf
import tensorflow.feature_column as fc
import csv
import sys
import os
import pandas as pd
import numpy as np
import gzip

In [21]:

# file_path=os.path.join(DATASET_PATH,"ds3.csv")
BATCH_SIZE=32
STEPS=1000
NUM_EPOCHS=40
shuffle_buffer_size=70000
prefetch_buffer_size=32
num_classes= 62



In [22]:
def _parse_function(filename, label):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=1)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    #image = tf.image.resize_images(image, [64, 64])
    return image, label

In [81]:

def input_fn():
    mapping = pd.read_csv("train-labels.csv", header = None)
    
    filenames = list(mapping.iloc[:,0])
    labels = list(mapping.iloc[:,1])
    
    parse_fn = lambda f, l: _parse_function(f, l)
    
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.shuffle(len(filenames))
    dataset = dataset.map(parse_fn, num_parallel_calls=4)
    #dataset = dataset.map(train_preprocess, num_parallel_calls=4)
    dataset = dataset.repeat(5)
    dataset = dataset.batch(64)
    dataset = dataset.prefetch(1)
    
    
    iterator = dataset.make_one_shot_iterator()
    images, labels = iterator.get_next()
    #iterator = dataset.make_initializable_iterator()
    #images, labels = iterator.get_next()
    #iterator_init_op = iterator.initializer
    
    #inputs = {'images': images, 'labels': labels, 'iterator_init_op': iterator_init_op}
    return {"images": images}, labels

def test_input_fn():
    mapping = pd.read_csv("test-labels.csv", header = None)
    filenames = list(mapping.iloc[:,0])
    labels = list(mapping.iloc[:,1])
    
    parse_fn = lambda f, l: _parse_function(f, l)
    
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(parse_fn, num_parallel_calls=4)
    #dataset = dataset.repeat(2)
    dataset = dataset.batch(64)
    dataset = dataset.prefetch(1)
    
    
    iterator = dataset.make_one_shot_iterator()
    images, labels = iterator.get_next()
    #iterator = dataset.make_initializable_iterator()
    #images, labels = iterator.get_next()
    #iterator_init_op = iterator.initializer
    
    #inputs = {'images': images, 'labels': labels, 'iterator_init_op': iterator_init_op}
    return {"images": images}, labels



In [76]:
def build_dnn_classifier(hidden_units=None):
    feature_cols=[fc.numeric_column("images", shape=[784])]
    estimator = tf.estimator.DNNClassifier(feature_columns=feature_cols,hidden_units = hidden_units,activation_fn=tf.nn.relu,n_classes=num_classes,model_dir="./model_checkpoints")
    return estimator  

def train_dnn_classifier(steps,hidden_units=None):
    estimator = build_dnn_classifier(hidden_units)
    print("Model Built!")
    estimator.train(input_fn = input_fn,steps=steps)
    print("Finished Training!")
    return estimator


In [34]:
est=train_dnn_classifier(2000, hidden_units=[512,256,128,64])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022856737D68>, '_is_chief': True, '_session_config': None, '_save_checkpoints_steps': None, '_train_distribute': None, '_task_id': 0, '_device_fn': None, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_service': None, '_evaluation_master': '', '_task_type': 'worker', '_keep_checkpoint_max': 5, '_master': '', '_save_summary_steps': 100, '_model_dir': './model_checkpoints'}
Model Built!
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_checkpoints\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running 

In [54]:
def predict():
    preds=[]
    predictions = est.predict(input_fn=test_input_fn)
    for prediction in predictions:
        preds.append(prediction["class_ids"][0])
    return preds


In [50]:
preds=predict()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_checkpoints\model.ckpt-7000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [51]:
# test-labels.csv - contains the mappings of the filenames to thier labels# test-l 
mapping = pd.read_csv("test-labels.csv", header = None)

# actual labels
actual = list(mapping.iloc[:,1])

In [61]:
print(len(actual),len(preds))

15000 15000


In [87]:
def my_accuracy():
    act = tf.placeholder(tf.int64, shape = [15000])
    pred = tf.placeholder(tf.int64, shape = [15000])
    
    acc = tf.metrics.accuracy(labels=act, predictions=pred)[1]
    with tf.Session() as sess:
        #sess.graph._unsafe_unfinalize()
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        res=sess.run([acc], feed_dict={act: actual, pred: preds})[0]
    return res

def my_precision():
    act = tf.placeholder(tf.int64, shape = [15000])
    pred = tf.placeholder(tf.int64, shape = [15000])
    
    prec = tf.metrics.precision(labels=act, predictions=pred)[1]
    with tf.Session() as sess:
        #sess.graph._unsafe_unfinalize()
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        res=sess.run([prec], feed_dict={act: actual, pred: preds})[0]
    return res


def my_recall():
    act = tf.placeholder(tf.int64, shape = [15000])
    pred = tf.placeholder(tf.int64, shape = [15000])
    
    rec = tf.metrics.recall(labels=act, predictions=pred)[1]
    with tf.Session() as sess:
        #sess.graph._unsafe_unfinalize()
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        res=sess.run([rec], feed_dict={act: actual, pred: preds})[0]
    return res

def my_confusion_matrix():
    act = tf.placeholder(tf.int64, shape = [15000])
    pred = tf.placeholder(tf.int64, shape = [15000])
    
    confmat = tf.confusion_matrix(labels=act, predictions=pred)[1]
    with tf.Session() as sess:
        #sess.graph._unsafe_unfinalize()
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        res=sess.run([confmat], feed_dict={act: actual, pred: preds})[0]
    return res

def my_recall_class():
    acts_k = tf.placeholder(tf.int64, shape = [None])
    preds_k = tf.placeholder(tf.int64, shape = [None])
    recall_k, recall_op_k = tf.metrics.recall(labels=acts_k, predictions=preds_k)
    
    act_pred={n:[] for n in range(62)}

    
    for i in range(15000):#for all the test images do the following
        k=actual[i]
        act_pred[k].append((actual[i],preds[i]))
    
    
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        for k in range(62):
            temp = act_pred[k]
            temp2 = list(zip(*temp))
            actual_k=list(temp2[0])
            pred_k=list(temp2[1])
            print(k, "   : ", end = " ")
            print(sess.run([recall_k, recall_op_k], feed_dict={acts_k: actual_k, preds_k: pred_k}))

            
def my_precision_class():
    acts_k = tf.placeholder(tf.int64, shape = [None])
    preds_k = tf.placeholder(tf.int64, shape = [None])
    precision_k, precision_op_k = tf.metrics.recall(labels=acts_k, predictions=preds_k)
    
    act_pred={n:[] for n in range(62)}

    
    for i in range(15000):#for all the test images do the following
        k=actual[i]
        act_pred[k].append((actual[i],preds[i]))
    
    
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        for k in range(62):
            temp = act_pred[k]
            temp2 = list(zip(*temp))
            actual_k=list(temp2[0])
            pred_k=list(temp2[1])
            print(k, "   : ", end = " ")
            print(sess.run([precision_k, precision_op_k], feed_dict={acts_k: actual_k, preds_k: pred_k}))

            
def evaluate_metrics():
    acc=my_accuracy()
    prec=my_precision()
    rec=my_recall()
    confmat=my_confusion_matrix()
    
    print("METRICS ARE:")
    metrics={"accuracy":acc,"precsion":prec,"recall":rec,"confusion_matrix":confmat}
    print(metrics)
    
    print("PER CLASS RECALL IS")
    my_recall_class() 
   
    print("PER CLASS PRECISION IS")
    my_precision_class() 

#     Try the add_metrics function later!
#     estimator=est
#     estimator = tf.contrib.estimator.add_metrics(estimator, my_accuracy)
#     estimator = tf.contrib.estimator.add_metrics(estimator, my_precision)
#     estimator = tf.contrib.estimator.add_metrics(estimator, my_recall)
#     estimator = tf.contrib.estimator.add_metrics(estimator, my_confusion_matrix)
#     #estimator = tf.contrib.estimator.add_metrics(estimator, my_recall_class)  
#     metrics=estimator.evaluate(input_fn=input_fn)
#     return metrics

In [88]:
metrics=evaluate_metrics()

METRICS ARE:
{'confusion_matrix': array([  0, 734,   2,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,
         0,   0,   1,   0,   0,  24,   0,   0,   1,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   2,   1,   1,  89,   0,   0,   0,   0,
         0,   1,   0,   1,   0,   0,   0,   0,   0,   0]), 'accuracy': 0.7974, 'precsion': 0.98813474, 'recall': 0.9801051}
PER CLASS RECALL IS
0    :  [0.0, 0.0]
1    :  [0.0, 1.0]
2    :  [1.0, 1.0]
3    :  [1.0, 0.99958557]
4    :  [0.99958557, 0.9996846]
5    :  [0.9996846, 0.9997374]
6    :  [0.9997374, 0.9997792]
7    :  [0.9997792, 0.9998129]
8    :  [0.9998129, 0.9996701]
9    :  [0.9996701, 0.99970603]
10    :  [0.99970603, 0.9997118]
11    :  [0.9997118, 0.9991464]
12    :  [0.9991464, 0.9990338]
13    :  [0.9990338, 0.99754435]
14    :  [0.99754435, 0.9975829]
15    :  [0.9975829, 0.99765074]
16    :  [0.99765074, 0.99740696]
17    :  [0.99740696, 0.997426]
1